# Nombre de cas positifs COVID 19 en France par jour et par département

## Imports

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import os

## Téléchargement et sauvegarde du fichier csv des nouvelles contaminations (les données sont mises à jour au quotidien)

In [ ]:
url = "https://www.data.gouv.fr/fr/datasets/r/426bab53-e3f5-4c6a-9d54-dba4442b3dbc"
file_name = "incidence.csv"

In [ ]:
response = requests.get(url)
if response.status_code == 200:
    print("Requête OK")

### Écriture du fichier sur le disque

In [ ]:
with open(file_name, "w") as csv_file:
    csv_file.write(response.text)

### Fonction qui va créer la DataFrame propre

In [ ]:
def get_data():
    data = pd.read_csv(file_name, sep=";", dtype={'dep': str})

    # mettre la colonne jour en datetime
    data.jour = pd.to_datetime(data.jour)
    
    columns_to_numerize = ['pop', 'P', 'T', 'Ti']
  
    
    for col in columns_to_numerize:
        data[col] = data[col].apply(lambda s: float(f"{s[:-3]}.{s[-2:]}"))

    # Garder uniquement cl_age90 0 qui est la somme des positifs de toutes les classes d'âge
    data = data.loc[data.cl_age90 == 0]
    return data.reset_index().drop(columns=['index', 'cl_age90'])

In [ ]:
data = get_data()

In [ ]:
data.sample(5)

In [ ]:
data[data.dep == '69']

## On regroupe par département et par jour

In [ ]:
sum_by_day = data.groupby(["dep", "jour"]).sum().drop(columns=["pop"])

## Graphe. Changer le numéro de département pour celui que vou voulez.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

sum_by_day.loc["75", "P"].plot(c='r', ax=ax)
sum_by_day.loc["69", "P"].plot(c='b', ax=ax);

## On efface les fichiers csv dans le dossier

In [ ]:
csv_files = [f for f in os.listdir() if f.endswith('csv')]

for f in csv_files:
    os.remove(f)